### 寻找最优参数

In [ ]:
import pickle
import warnings
warnings.filterwarnings('ignore')

# 读取最优特征数据集
with open('../data/processed/experiment_data_refined.pkl', 'rb') as f:
    experiment_data_refined = pickle.load(f)

print("The filtered feature data has been successfully loaded from experiment_data_refined.pkl.")

The filtered feature data has been successfully loaded from experiment_data_refined.pkl.


In [ ]:
# 轻量级参数优化版本
# 最优参数组合将在下方代码运行遍历后，针对每个窗口自动打印输出。

import itertools
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, GRU, MultiHeadAttention, LayerNormalization, Dense, Concatenate, Dropout, SpatialDropout1D, Lambda
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
import warnings

warnings.filterwarnings("ignore")
tf.get_logger().setLevel('ERROR')

# 初始化全局存储结构
if 'all_results' not in globals():
    all_results = []
if 'all_errors' not in globals():
    all_errors = []

model_name = "TimesNet-GRU-MHA"
current_results = {}

# 清理逻辑：覆盖当前模型的历史运行记录，防止重复 append 污染最终的 DM 检验
all_results = [res for res in all_results if res['model'] != model_name]
all_errors = [err for err in all_errors if err['model'] != model_name]

# 定义轻量级搜索空间
search_space = {
    'lr': [0.001, 0.0008, 0.0005, 0.0003],
    'gru_units': [24, 32, 48],
    'num_heads': [2, 4, 5],
    'dropout': [0.05, 0.1, 0.15]
}

for w, data in experiment_data_refined.items():
    X_train_3d = data['X_train']
    X_test_3d = data['X_test']
    
    y_train = data['y_train'].flatten()
    y_true = data['y_test'].flatten()
    y_pred_baseline = np.zeros_like(y_true)
    
    N_tr, L, D = X_train_3d.shape
    
    best_r2_os = -np.inf
    best_da = -np.inf
    best_results_dict = None
    best_error_dict = None
    best_params = None
    
    # 简单 for 循环遍历组合
    keys, values = zip(*search_space.items())
    for param_vals in itertools.product(*values):
        lr, units, heads, dp = param_vals
        
        K.clear_session()
        tf.random.set_seed(42)
        
        # 构建模型
        inputs = Input(shape=(L, D))
        
        k_sizes = list(set([min(k, L) for k in [2, 3, 5]]))
        convs = [Conv1D(filters=16, kernel_size=k, padding='same', activation='relu', 
                        kernel_regularizer=regularizers.l2(1e-5))(inputs) for k in k_sizes]
        x = Concatenate()(convs) if len(convs) > 1 else convs[0]
        x = SpatialDropout1D(dp)(x) 
        
        x_gru = GRU(units, return_sequences=True)(x)
        x_gru = Dropout(dp)(x_gru) 
        
        attn_out = MultiHeadAttention(num_heads=heads, key_dim=16, dropout=dp)(x_gru, x_gru)
        x = LayerNormalization()(x_gru + attn_out)
        
        x = Lambda(lambda seq: seq[:, -1, :])(x)
        
        initializer = tf.keras.initializers.VarianceScaling(scale=0.1, mode='fan_in', distribution='uniform')
        outputs = Dense(1, kernel_initializer=initializer, kernel_regularizer=regularizers.l2(1e-5))(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer=Adam(learning_rate=lr), loss='mse')
        
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)
        ]
        
        model.fit(X_train_3d, y_train, epochs=150, batch_size=32, 
                  validation_split=0.1, verbose=0, shuffle=False, callbacks=callbacks)
        
        # 预测与微调
        y_pred = model.predict(X_test_3d, verbose=0).flatten()
        epsilon = 5e-5
        y_pred_adj = np.where(np.abs(y_pred) < epsilon, 0, y_pred)
        
        # 计算当前指标
        rmse = np.sqrt(np.mean((y_true - y_pred)**2))
        mae = np.mean(np.abs(y_true - y_pred))
        r2_os = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - y_pred_baseline)**2))
        da = np.mean(np.sign(y_true) == np.sign(y_pred_adj)) * 100
        
        # 优选逻辑：优先 R2_OS，差异极小则比较 DA
        if (r2_os > best_r2_os + 1e-4) or (abs(r2_os - best_r2_os) <= 1e-4 and da > best_da):
            best_r2_os = r2_os
            best_da = da
            best_params = {'lr': lr, 'gru_units': units, 'num_heads': heads, 'dropout': dp}
            
            e_t = y_true - y_pred
            e0_t = y_true - y_pred_baseline
            
            best_results_dict = {
                "model": model_name,
                "window": w,
                "RMSE": rmse,
                "MAE": mae,
                "R2_OS": r2_os,
                "DA": da
            }
            best_error_dict = {
                "model": model_name,
                "window": w,
                "errors": e_t,
                "benchmark_errors": e0_t
            }

    # 仅保留并打印该窗口下最佳参数组合
    print(f"Window {w} 最佳参数组合: {best_params}")
    all_results.append(best_results_dict)
    all_errors.append(best_error_dict)
    current_results[w] = best_results_dict

# ---------------- 输出格式模拟 ----------------
print("-" * 110)
# 表头第一行：窗口
header_win = f"{'Model':<20}"
# 表头第二行：指标
header_met = f"{'':<20}"
# 数据行
row_data = f"{model_name:<20}"

for w in experiment_data_refined.keys():
    header_win += f"{f'Window = {w}':^30}"
    header_met += f"{'RMSE':>7} {'MAE':>7} {'R2_OS':>7} {'DA (%)':>7}  "
    
    res = current_results[w]
    row_data += f"{res['RMSE']:>7.4f} {res['MAE']:>7.4f} {res['R2_OS']:>7.4f} {res['DA']:>7.2f}  "

print(header_win)
print(header_met)
print("-" * 110)
print(row_data)
print("-" * 110)